In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes
    
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
# set_log_level("ERROR")

In [2]:
# data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
data_location = '../../../neuralprophet-data/datasets/'

sf_load_df = pd.read_csv(data_location +  'energy/SF_hospital_load.csv')

In [3]:
sf_pv_df.head(3)

NameError: name 'sf_pv_df' is not defined

In [ ]:
m = NeuralProphet(
    # n_lags=24,
    # ar_sparsity=0.5,
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=8,
    seasonality_mode='multiplicative',
    learning_rate=0.03,
    #num_hidden_layers = 2,
    #d_hidden=20,
)
sf_pv_df_train, sf_pv_df_test = m.split_df(sf_pv_df, freq='H', valid_p = 1.0/12)

metrics = m.fit(sf_pv_df, freq='H', validation_df=sf_pv_df_test, plot_live_loss=True)

In [ ]:
metrics.tail(1)

In [ ]:
forecast = m.predict(sf_pv_df_train)
fig = m.plot(forecast)

In [ ]:
forecast = m.predict(sf_pv_df_test)
m = m.highlight_nth_step_ahead_of_each_forecast(1)
fig = m.plot(forecast)

In [ ]:
fig_comp = m.plot_components(forecast)

In [ ]:
fig_param = m.plot_parameters()


In [ ]:
fig_prediction = m.plot(forecast[-48:])


There is something to consider here, if we consider a neural network with at least one hidden layer. For a high enough learning rate (probably > 0.1), the gradient seems to vanish and forces the AR net output to 0.  

From below, we can see that the forecast output has a strange behavior. it looks like the output is exactly the sum of the weekly, daily seasonlality with trend. With no noise and changes at all.

In [ ]:
m = NeuralProphet(
    n_lags=24,
    ar_sparsity=0.5,
    num_hidden_layers = 2,
    d_hidden=20,
    learning_rate = 0.1
)
metrics = m.fit(df, freq='H', valid_p = 0.2)

In [ ]:
future = m.make_future_dataframe(df_val, n_historic_predictions=True)
forecast = m.predict(future)
fig = m.plot(forecast)

An easy fix is to set the learning rate at a low enough value.

In [ ]:
m = NeuralProphet(
    n_lags=24,
    ar_sparsity=0.5,
    num_hidden_layers = 2,
    d_hidden=20,
    learning_rate = 0.001
)
metrics = m.fit(df, freq='H', valid_p = 0.2)

In [ ]:
future = m.make_future_dataframe(df_val, n_historic_predictions=True)
forecast = m.predict(future)
fig = m.plot(forecast)

## Example on load forecasting

We can train a forecaster on another commom energy problem. In this, case we are training a 1-step ahead forecaster to predict the electricity consumption of a building. We use a similar NeuralProphet model as we used for the PV forecasting.

In [ ]:
m = NeuralProphet(
    n_lags=24,
    ar_sparsity=0.5,
    num_hidden_layers = 2,
    d_hidden=20,
    learning_rate=0.001
)

sf_load_df_train, sf_load_df_test = m.split_df(sf_load_df, freq='H', valid_p = 0.10)

metrics = m.fit(sf_load_df_train, freq='H', validation_df=sf_load_df_test, plot_live_loss=True)

In [ ]:
df_train, df_val = m.split_df(df, freq='H',valid_p=0.2)
m.test(df_val)

In [ ]:
forecast = m.predict(sf_load_df)
fig = m.plot(forecast)

In [ ]:
m = m.highlight_nth_step_ahead_of_each_forecast(1)
fig_param = m.plot_parameters()

In [ ]:
forecast = m.predict(sf_load_df_test)
fig = m.plot(forecast)

In [ ]:
fig_comp = m.plot_components(forecast)


In [ ]:
fig_prediction = m.plot(forecast[-48:])
